In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

In [ ]:
base = 'C:\\Users\\zshal\\Documents\\Pytorch\\Images\\Images\\v3'
data = ImageFolder(root=base, transform=ToTensor())
lengths = ((int)(len(data)*0.8), (int)(len(data)*0.2))
trainsplit, testsplit = torch.utils.data.random_split(data, lengths)

trainimages = DataLoader(trainsplit, batch_size=4, shuffle=True, num_workers=0)
testimages = DataLoader(testsplit, batch_size=4, shuffle=False, num_workers=0)


transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5, 0.5, 0.5),
                                (0.5, 0.5, 0.5))])



# classes = ('plane', 'car', 'bird', 'cat',
#          'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

classes = ('pothole', 'no pothole')

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 3)
        self.pool = nn.MaxPool2d(2, 2)
        #self.conv2 = nn.Conv2d(12, 12, 3)
        #self.fc1 = nn.Linear(555984, 120)
        self.fc1 = nn.Linear( 12*399*471, 360)
        self.fc2 = nn.Linear( 360, 120)
        self.fc3 = nn.Linear(120, 84)
        self.fc4 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        #x = self.pool(F.relu(self.conv2(x)))
        #x = x.view(-1, 12*198*234)
        x = x.view(-1, 12*399*471)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
torch.manual_seed(0)
net = Net()
#net.to(device)
model = NeuralNetClassifier(
    net,
    max_epochs=10,
    lr=0.1,
    optimizer=torch.optim.Adam,
    train_split=None,
    #device=device,
)

params = {
    'lr': [0.01, 0.02],
    'max_epochs': [10, 20],
    #'module__num_units': [10, 20],
}

model_CV = model_selection.GridSearchCV(model, params, 
    iid=False, 
    scoring='neg_log_loss',
    cv=5,
    verbose=1,
    n_jobs=1,
    return_train_score=True,
    )

In [ ]:
model_CV.fit(trainsplit, y=None)

model_CV.save_params(f_params='skorchmodelCV.pkl')


y_pred = model.predict(testimages[0])
accuracy_score(testimages[0], y_pred)